# Flask with Database
---

## 🌐 Overview

* Flask does **not include a database by default**.

* You can integrate **any database**:

  1. **SQLite** – lightweight, file-based (good for small projects)
  2. **MySQL/PostgreSQL** – for production apps
  3. **SQLAlchemy** – ORM (Object-Relational Mapper) for working with databases using Python classes

* Common approaches:

  * **Direct SQL queries** (sqlite3, pymysql)
  * **Using Flask extensions** like **Flask-SQLAlchemy** (recommended)

---

## ✅ 1️⃣ Using SQLite with Flask (Direct Approach)

**Folder structure:**

```
project/
│   app.py
└───templates/
      users.html
```

**app.py**

```python
from flask import Flask, render_template
import sqlite3

app = Flask(__name__)

# Create/connect to database and create table
def init_db():
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("""CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT NOT NULL,
                    email TEXT NOT NULL
                )""")
    conn.commit()
    conn.close()

@app.route("/add_user/<name>/<email>")
def add_user(name, email):
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("INSERT INTO users (name, email) VALUES (?, ?)", (name, email))
    conn.commit()
    conn.close()
    return f"User {name} added!"

@app.route("/users")
def users():
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("SELECT * FROM users")
    users_list = c.fetchall()
    conn.close()
    return render_template("users.html", users=users_list)

if __name__ == "__main__":
    init_db()
    app.run(debug=True)
```

**templates/users.html**

```html
<!DOCTYPE html>
<html>
<head>
    <title>Users</title>
</head>
<body>
    <h1>All Users</h1>
    <ul>
        {% for user in users %}
            <li>{{ user[1] }} ({{ user[2] }})</li>
        {% endfor %}
    </ul>
</body>
</html>
```

**Usage:**

* Visit `/add_user/John/john@example.com` → adds user
* Visit `/users` → lists all users

---

## ✅ 2️⃣ Using Flask-SQLAlchemy (ORM)

**Install SQLAlchemy**

```bash
pip install Flask-SQLAlchemy
```

**app.py**

```python
from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
db = SQLAlchemy(app)

# Model
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(50), nullable=False)
    email = db.Column(db.String(120), nullable=False)

    def __repr__(self):
        return f"<User {self.name}>"

@app.before_first_request
def create_tables():
    db.create_all()

@app.route("/add_user/<name>/<email>")
def add_user(name, email):
    new_user = User(name=name, email=email)
    db.session.add(new_user)
    db.session.commit()
    return f"User {name} added!"

@app.route("/users")
def users():
    users_list = User.query.all()
    return render_template("users.html", users=users_list)

if __name__ == "__main__":
    app.run(debug=True)
```

**templates/users.html** (same as above)

```html
<ul>
    {% for user in users %}
        <li>{{ user.name }} ({{ user.email }})</li>
    {% endfor %}
</ul>
```

**Advantages of SQLAlchemy:**

* Work with **Python objects** instead of raw SQL
* Easy **CRUD operations**
* Cleaner and maintainable code

---

## 🔎 Key Points

1. **SQLite** is great for small projects or learning.
2. **Flask-SQLAlchemy** simplifies database management with ORM.
3. Always **create models (classes)** for each table.
4. Use **`db.session.add()` and `db.session.commit()`** to save changes.
5. Combine databases with **forms**, **template rendering**, and **Flask-WTF** for a full dynamic app.

---

